In [1]:
!pip install dash_bootstrap_components

In [ ]:
import dash
from dash import dcc, html, dash_table
from dash.dependencies import Input, Output
import dash_bootstrap_components as dbc
import plotly.graph_objects as go
import pandas as pd
from datetime import datetime as dt, timedelta

# --- 1. Data Loading and Preparation ---

# Load the dataset
try:
    df = pd.read_csv(r"/content/global_indices_master_3y_daily.csv")
    df['Date'] = pd.to_datetime(df['Date'])
    df.set_index('Date', inplace=True)
except FileNotFoundError:
    print("Error: 'global_indices_master_3y_daily.csv' not found.")
    print("Please run the data fetching script first and ensure the CSV is in the same directory.")
    exit()

# Interpolate missing values for each index
df = df.apply(lambda x: x.interpolate())


# Categorize indices by region for the tabs.
# This is based on the original dictionary structure.
REGIONS = {
    "Global": ["Dow Jones Global Index", "FTSE All-World", "S&P Global 100", "S&P Global 1200", "MSCI World", "MSCI EAFE"],
    "Americas": ["S&P 500 (USA)", "Dow Jones Industrial Average (USA)", "NASDAQ Composite (USA)", "Russell 2000 (USA)", "CBOE Volatility Index (VIX)", "MERVAL (Argentina)", "Bovespa Index (Brazil)", "S&P/TSX Composite (Canada)", "IPSA (Chile)", "COLCAP (Colombia)", "IPC (Mexico)", "S&P/BVL Peru General", "S&P Latin America 40"],
    "Asia-Pacific": ["S&P/ASX 200 (Australia)", "All Ordinaries (Australia)", "SSE Composite (China)", "CSI 300 (China)", "SZSE Component (China)", "Hang Seng (Hong Kong)", "Nifty 50 (India)", "BSE SENSEX (India)", "Jakarta Composite (Indonesia)", "TA-125 (Israel)", "Nikkei 225 (Japan)", "TOPIX (Japan)", "KLCI (Malaysia)", "S&P/NZX 50 (New Zealand)", "PSEi (Philippines)", "Tadawul All-Share (Saudi Arabia)", "Straits Times Index (Singapore)", "KOSPI (South Korea)", "TAIEX (Taiwan)", "SET Index (Thailand)", "VN-Index (Vietnam)", "S&P Asia 50"],
    "Europe & Africa": ["ATX (Austria)", "BEL 20 (Belgium)", "OMX Copenhagen 25 (Denmark)", "EGX 30 (Egypt)", "OMX Helsinki 25 (Finland)", "CAC 40 (France)", "DAX (Germany)", "MDAX (Germany)", "TecDAX (Germany)", "Athex Composite (Greece)", "ISEQ 20 (Ireland)", "FTSE MIB (Italy)", "AEX (Netherlands)", "OBX (Norway)", "WIG20 (Poland)", "PSI 20 (Portugal)", "MOEX (Russia)", "JSE Top 40 (South Africa)", "IBEX 35 (Spain)", "OMX Stockholm 30 (Sweden)", "Swiss Market Index (SMI)", "BIST 100 (Turkey)", "FTSE 100 (UK)", "FTSE 250 (UK)", "EURO STOXX 50", "STOXX Europe 600"],
}
# Filter for columns that actually exist in our dataframe
for region, indices in REGIONS.items():
    REGIONS[region] = [idx for idx in indices if idx in df.columns]

# --- 2. Initialize the Dash App ---
app = dash.Dash(__name__, external_stylesheets=[dbc.themes.MINTY]) # Use a clean bootstrap theme

# --- 3. App Layout ---
app.layout = dbc.Container([
    # Page Header
    dbc.Row(dbc.Col(html.H1("Markets Data", className="text-center my-4"), width=12)),

    dbc.Row([
        # Main content area (Chart and Table)
        dbc.Col([
            html.H4("Equity Indices"),
            # Tabs for region selection
            dbc.Tabs(id="region-tabs", active_tab="Americas", children=[
                dbc.Tab(label=region, tab_id=region) for region in REGIONS.keys()
            ]),

            # Dropdown for selecting indices
            dcc.Dropdown(
                id='index-dropdown',
                multi=True,
                value=['S&P 500 (USA)', 'NASDAQ Composite (USA)'], # Default selection
                className="my-3"
            ),

            # Time range slider
            dcc.Slider(
                id='time-range-slider',
                min=0,
                max=3, # Representing years (0=last week, 1=last month, 2=last year, 3=all time)
                step=1,
                marks={0: 'Last Week', 1: 'Last Month', 2: 'Last Year', 3: 'All Time'},
                value=3, # Default to All Time
                className="my-3"
            ),

            # Graph to display performance
            dcc.Graph(id='performance-graph', style={'height': '600px'}), # Added style to constrain height

            # Data table for summary stats
            dash_table.DataTable(
                id='summary-table',
                style_cell={'textAlign': 'left', 'fontFamily': 'sans-serif'},
                style_header={'fontWeight': 'bold'},
                style_data_conditional=[{'if': {'row_index': 'odd'}, 'backgroundColor': 'rgb(248, 248, 248)'}]
            )
        ], width=9),

        # Sidebar with "Explore our tools"
        dbc.Col([
            html.H4("Explore our tools"),
            dbc.Card([
                dbc.CardBody([
                    html.H5("Growth Calculator", className="card-title"),
                    html.P("Explore the time value of money, the impact of regular contributions, and the power of saving.", className="card-text")
                ])
            ], className="mb-3"),
            dbc.Card([
                dbc.CardBody([
                    html.H5("Alerts", className="card-title"),
                    html.P("Create detailed alerts and get notified the moment an event happens.", className="card-text")
                ])
            ], className="mb-3"),
            dbc.Card([
                dbc.CardBody([
                    html.H5("Watchlists", className="card-title"),
                    html.P("Monitor a select list of assets.", className="card-text")
                ])
            ])
        ], width=3)
    ])
], fluid=True)


# --- 4. Callbacks for Interactivity ---

# Callback 1: Update dropdown options based on the selected region tab
@app.callback(
    Output('index-dropdown', 'options'),
    Input('region-tabs', 'active_tab')
)
def update_dropdown_options(selected_region):
    return [{'label': i, 'value': i} for i in REGIONS[selected_region]]

# Callback 2: Update the graph and table based on dropdown selection and time range
@app.callback(
    Output('performance-graph', 'figure'),
    Output('summary-table', 'data'),
    Output('summary-table', 'columns'),
    Input('index-dropdown', 'value'),
    Input('time-range-slider', 'value')
)
def update_graph_and_table(selected_indices, selected_time_range):
    if not selected_indices:
        return go.Figure(), [], []

    # Determine the date range based on the slider value
    end_date = df.index.max()
    if selected_time_range == 0: # Last Week
        start_date = end_date - timedelta(weeks=1)
    elif selected_time_range == 1: # Last Month
        start_date = end_date - timedelta(days=30) # Approximation of a month
    elif selected_time_range == 2: # Last Year
        start_date = end_date - timedelta(days=365) # Approximation of a year
    else: # All Time
        start_date = df.index.min()

    # Filter the dataframe based on the selected date range
    filtered_df = df.loc[start_date:end_date]

    # --- Create Performance Graph ---
    fig = go.Figure()
    for index_name in selected_indices:
        # Normalize the data to show percentage change since the start of the filtered range
        series = filtered_df[index_name].dropna()
        if len(series) < 2:
             continue
        normalized_series = (series / series.iloc[0] - 1) * 100
        fig.add_trace(go.Scatter(x=normalized_series.index, y=normalized_series,
                                 mode='lines', name=index_name))

    fig.update_layout(
        title=f"Normalized Performance ({'Last Week' if selected_time_range == 0 else 'Last Month' if selected_time_range == 1 else 'Last Year' if selected_time_range == 2 else 'All Time'})",
        yaxis_title="Percentage Change (%)",
        template='plotly_white',
        legend=dict(orientation="h", yanchor="bottom", y=1.02, xanchor="right", x=1)
    )

    # --- Create Summary Table Data ---
    table_data = []
    # ~252 trading days in a year
    trading_days_in_year = 252

    for index_name in selected_indices:
        series = filtered_df[index_name].dropna()
        if len(series) < 2:
            continue

        last_price = series.iloc[-1]
        prev_price = series.iloc[-2]
        change = last_price - prev_price
        pct_change = (change / prev_price) * 100

        # Calculate 52-week range on the last year of data (within the filtered range)
        # Ensure there is enough data for a 52-week range calculation
        if len(series) >= trading_days_in_year:
             last_year_data = series.tail(trading_days_in_year)
             low_52_week = last_year_data.min()
             high_52_week = last_year_data.max()
             range_52_week_str = f"{low_52_week:,.2f} - {high_52_week:,.2f}"
        else:
            range_52_week_str = "Insufficient data for 52-week range"


        table_data.append({
            "Index": index_name,
            "Last": f"{last_price:,.2f}",
            "Change": f"{change:+.2f} ({pct_change:+.2f}%)",
            "52-Week Range": range_52_week_str
        })

    table_columns = [
        {"name": "Index", "id": "Index"},
        {"name": "Last", "id": "Last"},
        {"name": "Today's Change", "id": "Change"},
        {"name": "52-Week Range", "id": "52-Week Range"}
    ]

    return fig, table_data, table_columns


# --- 5. Run the App ---
if __name__ == '__main__':
    app.run(debug=True)

ModuleNotFoundError: No module named 'dash_bootstrap_components'